In [ ]:
# ### 코랩용
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/Othercomputers/내 노트북/gdrive/dppos/cpe')
# !pip uninstall -y gym
# !pip install gymnasium stable-baselines3


텐서보드 log 확인

```
tensorboard --logdir ./ppo_tensorboard_logs/
```

[http://localhost:6006](http://localhost:6006)

In [ ]:
import time
from drivingppo.common import set_seed
from drivingppo.environment import WorldEnv
from drivingppo.model import train_start, train_resume
from world_samples import gen_0, gen_1, gen_2

prefix = 'cpe'
t0 = time.strftime('%y%m%d-%H%M')

def a(seed):

    set_seed(seed)
    train_start(
        save_path=f'{prefix}-{seed}--0',
        gen_env=lambda: WorldEnv(world_generator=gen_0),
        tb_log=True,
        run_name=f'{prefix}-{t0}-{seed}',
        steps=2_0000,
        lr=5e-4,
        gamma=0.95,
        ent_coef=0.02,
    )

    set_seed(seed)
    train_resume(
        model=    f'{prefix}-{seed}--0',
        save_path=f'{prefix}-{seed}--1',
        gen_env=lambda: WorldEnv(world_generator=gen_1),
        tb_log=True,
        run_name=f'{prefix}-{t0}-{seed}',
        steps=20_0000,
        lr=3e-4,
        gamma=0.99,
        ent_coef=0.02,
    )

    set_seed(seed)
    train_resume(
        model=    f'{prefix}-{seed}--1',
        save_path=f'{prefix}-{seed}--2',
        gen_env=lambda: WorldEnv(world_generator=gen_2),
        tb_log=True,
        run_name=f'{prefix}-{t0}-{seed}',
        steps=80_0000,
        lr=3e-4,
        gamma=0.996,
    )

a(100)
a(200)
a(300)
a(400)
a(500)

| `log_std` | 실제 표준편차 σ | 탐색 수준 | 비고 |
| --- | --- | --- | --- |
| 0 | 1.0 | 매우 높음 | 행동 공간 전체를 휘젓고 다니는 수준. 초기 학습 시 강한 탐색이 필요할 때 사용. |
| -0.5 | ≈ 0.6 | 높음 | 행동의 변동폭이 큼. 기존 정책이 굳었을 때 강제 리셋용으로 권장되는 값. |
| -1 | ≈ 0.37 | 보통 | 안정적인 주행과 탐색이 공존하는 수준. |
| -2 | ≈ 0.13 | 낮음 | 어느 정도 최적화가 진행된 상태. 정밀한 제어를 시작함. |
| \~-3.0 | \< 0.05 | 거의 없음 | 수렴 완료 단계. 거의 결정론적(Deterministic)인 행동을 보임. |